# Identifying safe loans with decision trees

The [LendingClub](https://www.lendingclub.com/) is a peer-to-peer leading company that directly connects borrowers and potential lenders/investors. In this notebook, we will build a classification model to predict whether or not a loan provided by LendingClub is likely to [default](https://en.wikipedia.org/wiki/Default_%28finance%29).

In this notebook we will use data from the LendingClub to predict whether a loan will be paid off in full or the loan will be [charged off](https://en.wikipedia.org/wiki/Charge-off) and possibly go into default. In this assignment we will:

  - Use SFrames to do some feature engineering.
  - Train a decision-tree on the LendingClub dataset.
  - Predict whether a loan will default along with prediction probabilities (on a validation set).
  - Train a complex tree model and compare it to simple tree model.

Let's get started!

## Fire up Turi Create

Make sure you have the latest version of Turi Create. If you don't find the decision tree module, then you would need to upgrade Turi Create using

```
   pip install turicreate --upgrade
```

In [1]:
import turicreate

# Load LendingClub dataset

We will be using a dataset from the [LendingClub](https://www.lendingclub.com/). A parsed and cleaned form of the dataset is availiable [here](https://github.com/learnml/machine-learning-specialization-private). Make sure you **download the dataset** before running the following command.

In [2]:
loans = turicreate.SFrame('../data/lending-club-data.sframe/')
loans.head(3)

Columns:
	id	int
	member_id	int
	loan_amnt	int
	funded_amnt	int
	funded_amnt_inv	int
	term	str
	int_rate	float
	installment	float
	grade	str
	sub_grade	str
	emp_title	str
	emp_length	str
	home_ownership	str
	annual_inc	int
	is_inc_v	str
	issue_d	str
	loan_status	str
	pymnt_plan	str
	url	str
	desc	str
	purpose	str
	title	str
	zip_code	str
	addr_state	str
	dti	float
	delinq_2yrs	int
	earliest_cr_line	str
	inq_last_6mths	int
	mths_since_last_delinq	int
	mths_since_last_record	int
	open_acc	int
	pub_rec	int
	revol_bal	int
	revol_util	float
	total_acc	int
	initial_list_status	str
	out_prncp	float
	out_prncp_inv	float
	total_pymnt	float
	total_pymnt_inv	float
	total_rec_prncp	float
	total_rec_int	float
	total_rec_late_fee	float
	recoveries	float
	collection_recovery_fee	float
	last_pymnt_d	str
	last_pymnt_amnt	float
	next_pymnt_d	str
	last_credit_pull_d	str
	collections_12_mths_ex_med	int
	mths_since_last_major_derog	str
	policy_code	int
	not_compliant	int
	status	str
	inactive_loans	int
	bad_loans	int
	emp_length_num	int
	grade_num	int
	sub_grade_num	float
	delinq_2yrs_zero	int
	pub_rec_zero	int
	collections_12_mths_zero	int
	short_emp	int
	payment_inc_ratio	float
	final_d	str
	last_delinq_none	int
	last_record_none	int
	last_major_derog_none	int

Rows: 3

Data:
+---------+-----------+-----------+-------------+-----------------+------------+
|    id   | member_id | loan_amnt | funded_amnt | funded_amnt_inv |    term    |
+---------+-----------+-----------+-------------+-----------------+------------+
| 1077501 |  1296599  |    5000   |     5000    |       4975      |  36 months |
| 1077430 |  1314167  |    2500   |     2500    |       2500      |  60 months |
| 1077175 |  1313524  |    2400   |     2400    |       2400      |  36 months |
+---------+-----------+-----------+-------------+-----------------+------------+
+----------+-------------+-------+-----------+-----------+------------+----------------+
| int_rate | installment | grade | sub_grade | emp_title | emp_length | home_ownership |
+----------+-------------+-------+-----------+-----------+------------+----------------+
|  10.65   |    162.87   |   B   |     B2    |           | 10+ years  |      RENT      |
|  15.27   |    59.83    |   C   |     C4    |   Ryder   |  < 1 year  |      RENT      |
|  15.96   |    84.33    |   C   |     C5    |           | 10+ years  |      RENT      |
+----------+-------------+-------+-----------+-----------+------------+----------------+
+------------+-----------------+-----------------+-------------+------------+
| annual_inc |     is_inc_v    |     issue_d     | loan_status | pymnt_plan |
+------------+-----------------+-----------------+-------------+------------+
|   24000    |     Verified    | 20111201T000000 |  Fully Paid |     n      |
|   30000    | Source Verified | 20111201T000000 | Charged Off |     n      |
|   12252    |   Not Verified  | 20111201T000000 |  Fully Paid |     n      |
+------------+-----------------+-----------------+-------------+------------+
+-------------------------------+-------------------------------+-----+
|              url              |              desc             | ... |
+-------------------------------+-------------------------------+-----+
| https://www.lendingclub.co... |   Borrower added on 12/22/... | ... |
| https://www.lendingclub.co... |   Borrower added on 12/22/... | ... |
| https://www.lendingclub.co... |                               | ... |
+-------------------------------+-------------------------------+-----+
[3 rows x 68 columns]

## Exploring some features

Let's quickly explore what the dataset looks like. First, let's print out the column names to see what features we have in this dataset.

In [3]:
len(loans.column_names()), loans.column_names()

(68,
 ['id',
  'member_id',
  'loan_amnt',
  'funded_amnt',
  'funded_amnt_inv',
  'term',
  'int_rate',
  'installment',
  'grade',
  'sub_grade',
  'emp_title',
  'emp_length',
  'home_ownership',
  'annual_inc',
  'is_inc_v',
  'issue_d',
  'loan_status',
  'pymnt_plan',
  'url',
  'desc',
  'purpose',
  'title',
  'zip_code',
  'addr_state',
  'dti',
  'delinq_2yrs',
  'earliest_cr_line',
  'inq_last_6mths',
  'mths_since_last_delinq',
  'mths_since_last_record',
  'open_acc',
  'pub_rec',
  'revol_bal',
  'revol_util',
  'total_acc',
  'initial_list_status',
  'out_prncp',
  'out_prncp_inv',
  'total_pymnt',
  'total_pymnt_inv',
  'total_rec_prncp',
  'total_rec_int',
  'total_rec_late_fee',
  'recoveries',
  'collection_recovery_fee',
  'last_pymnt_d',
  'last_pymnt_amnt',
  'next_pymnt_d',
  'last_credit_pull_d',
  'collections_12_mths_ex_med',
  'mths_since_last_major_derog',
  'policy_code',
  'not_compliant',
  'status',
  'inactive_loans',
  'bad_loans',
  'emp_length_num',


Here, we see that we have some feature columns that have to do with grade of the loan, annual income, home ownership status, etc. Let's take a look at the distribution of loan grades in the dataset.

In [4]:
loans['grade'].show()

Materializing SArray

We can see that over half of the loan grades are assigned values `B` or `C`. Each loan is assigned one of these grades, along with a more finely discretized feature called `sub_grade` (feel free to explore that feature column as well!). These values depend on the loan application and credit report, and determine the interest rate of the loan. More information can be found [here](https://www.lendingclub.com/public/rates-and-fees.action).

Now, let's look at a different feature.

In [5]:
loans['sub_grade'].show()

Materializing SArray

In [6]:
loans['home_ownership'].show()

Materializing SArray

This feature describes whether the loanee is mortaging, renting, or owns a home. We can see that a small percentage of the loanees own a home.

## Exploring the target column

The target column (label column) of the dataset that we are interested in is called `bad_loans`. In this column **1** means a risky (bad) loan **0** means a safe  loan.

In order to make this more intuitive and consistent with the lectures, we reassign the target to be:
* **+1** as a safe  loan, 
* **-1** as a risky (bad) loan. 

We put this in a new column called `safe_loans`.

In [7]:
## safe_loans =  1 => safe; safe_loans = -1 => risky
loans['safe_loans'] = loans['bad_loans'].apply(lambda x: +1 if x == 0 else -1)
loans = loans.remove_column('bad_loans')

Now, let us explore the distribution of the column `safe_loans`. This gives us a sense of how many safe and risky loans are present in the dataset.

In [8]:
loans['safe_loans'].show()

Materializing SArray

In [9]:
## num loans:
num_loans = len(loans)

## safe loans                                               |  bad loans:
round(loans['safe_loans'].value_counts()['count'][0] / num_loans, 4), round(loans['safe_loans'].value_counts()['count'][1] / num_loans, 4)

(0.8112, 0.1888)

We should have:
* Around 81% safe loans
* Around 19% risky loans

It looks like most of these loans are safe loans (thankfully). But this does make our problem of identifying risky loans challenging.

## Features for the classification algorithm

In this assignment, we will be using a subset of features (categorical and numeric). The features we will be using are **described in the code comments** below. If you are a finance geek, the [LendingClub](https://www.lendingclub.com/) website has a lot more details about these features.

In [10]:
features = ['grade',                     # grade of the loan
            'sub_grade',                 # sub-grade of the loan
            'short_emp',                 # one year or less of employment
            'emp_length_num',            # number of years of employment
            'home_ownership',            # home_ownership status: own, mortgage or rent
            'dti',                       # debt to income ratio
            'purpose',                   # the purpose of the loan
            'term',                      # the term of the loan
            'last_delinq_none',          # has borrower had a delinquincy
            'last_major_derog_none',     # has borrower had 90 day or worse rating
            'revol_util',                # percent of available credit being used
            'total_rec_late_fee',        # total late fees received to day
           ]

target = 'safe_loans'                    # prediction target (y) (+1 means safe, -1 is risky)

# Extract the feature columns and target column
loans = loans[features + [target]]
len(loans.column_names()), loans.column_names()

(13,
 ['grade',
  'sub_grade',
  'short_emp',
  'emp_length_num',
  'home_ownership',
  'dti',
  'purpose',
  'term',
  'last_delinq_none',
  'last_major_derog_none',
  'revol_util',
  'total_rec_late_fee',
  'safe_loans'])

What remains now is a **subset of features** and the **target** that we will use for the rest of this notebook. 

## Sample data to balance classes

As we explored above, our data is disproportionally full of safe loans.  Let's create two datasets: one with just the safe loans (`safe_loans_raw`) and one with just the risky loans (`risky_loans_raw`).

In [11]:
safe_loans_raw = loans[loans[target] == +1]
risky_loans_raw = loans[loans[target] == -1]
print("Number of safe loans  : %s" % len(safe_loans_raw))
print("Number of risky loans : %s" % len(risky_loans_raw))

Number of safe loans  : 99457
Number of risky loans : 23150


Now, write some code to compute below the percentage of safe and risky loans in the dataset and validate these numbers against what was given using `.show` earlier in the assignment:

In [12]:
def loan_ratios(safe_loans, risky_loans, num_loans=len(loans)):
  return (len(safe_loans) / num_loans, len(risky_loans) / num_loans)

p_safe_loans, p_risky_loans = loan_ratios(safe_loans_raw, risky_loans_raw)

print(f"Percentage of safe loans:  {p_safe_loans * 100.:1.2f}%") 
print(f"Percentage of risky loans: {p_risky_loans * 100.:3.2f}%")

Percentage of safe loans:  81.12%
Percentage of risky loans: 18.88%


One way to combat class imbalance is to undersample the larger class until the class distribution is approximately half and half. Here, we will undersample the larger class (safe loans) in order to balance out our dataset. This means we are throwing away many data points. We used `seed=1` so everyone gets the same results.

In [13]:
## Since there are fewer risky loans than safe loans, find the ratio of the sizes
## and use that percentage to undersample the safe loans.
percentage = float(len(risky_loans_raw) / len(safe_loans_raw))

risky_loans = risky_loans_raw
safe_loans = safe_loans_raw.sample(percentage, seed=1)  ## down sample safe loans 
loans_data = risky_loans.append(safe_loans)             ## Append the risky_loans to our ownsampled version of safe_loans

Now, let's verify that the resulting percentage of safe and risky loans are each nearly 50%.

In [14]:
np_safe_loans, np_risky_loans = loan_ratios(safe_loans, risky_loans, num_loans=len(loans_data))

print(f"Percentage of safe loans                 : {np_safe_loans * 100.:1.2f}%")
print(f"Percentage of risky loans                : {np_risky_loans * 100.:1.2f}%")
print(f"Total number of loans in our new dataset : {len(loans_data)}")

Percentage of safe loans                 : 50.22%
Percentage of risky loans                : 49.78%
Total number of loans in our new dataset : 46508


**Note:** There are many approaches for dealing with imbalanced data, including some where we modify the learning algorithm. These approaches are beyond the scope of this course, but some of them are reviewed in this [paper](http://ieeexplore.ieee.org/xpl/login.jsp?tp=&arnumber=5128907&url=http%3A%2F%2Fieeexplore.ieee.org%2Fiel5%2F69%2F5173046%2F05128907.pdf%3Farnumber%3D5128907 ). For this assignment, we use the simplest possible approach, where we subsample the overly represented class to get a more balanced dataset. In general, and especially when the data is highly imbalanced, we recommend using more advanced methods.

## Split data into training and validation sets

We split the data into training and validation sets using an 80/20 split and specifying `seed=1` so everyone gets the same results.

**Note**: In previous assignments, we have called this a **train-test split**. However, the portion of data that we don't train on will be used to help **select model parameters** (this is known as model selection). Thus, this portion of data should be called a **validation set**. Recall that examining performance of various potential models (i.e. models with different parameters) should be on validation set, while evaluation of the final selected model should always be on test data. Typically, we would also save a portion of the data (a real test set) to test our final model on or use cross-validation on the training set to select our final model. But for the learning purposes of this assignment, we won't do that.

In [15]:
train_data, validation_data = loans_data.random_split(.8, seed=1)

train_data.shape, validation_data.shape

((37224, 13), (9284, 13))

# Use decision tree to build a classifier

Now, let's use the built-in Turi Create decision tree learner to create a loan prediction model on the training data. (In the next assignment, you will implement your own decision tree learning algorithm.)  Our feature columns and target column have already been decided above. Use `validation_set=None` to get the same results as everyone else.

In [16]:
dt_model = turicreate.decision_tree_classifier.create(train_data,
                                                      validation_set=None,
                                                      target=target,
                                                      features=features,
                                                      verbose=False)
dt_model.summary()

Class                          : DecisionTreeClassifier

Schema
------
Number of examples             : 37224
Number of feature columns      : 12
Number of unpacked features    : 12
Number of classes              : 2

Settings
--------
Number of trees                : 1
Max tree depth                 : 6
Training time (sec)            : 0.0735
Training accuracy              : 0.6406
Training log_loss              : 0.6314
Training auc                   : 0.6927



## Building a smaller tree

Typically the max depth of the tree is capped at 6. However, such a tree can be hard to visualize graphically, and moreover, it may overfit..  Here, we instead learn a smaller model with **max depth of 2** to gain some intuition and to understand the learned tree more.

In [17]:
dt_small_model = turicreate.decision_tree_classifier.create(train_data,
                                                            validation_set=None,
                                                            target=target,
                                                            features=features,
                                                            max_depth=2,
                                                            verbose=False)
dt_small_model.summary()

Class                          : DecisionTreeClassifier

Schema
------
Number of examples             : 37224
Number of feature columns      : 12
Number of unpacked features    : 12
Number of classes              : 2

Settings
--------
Number of trees                : 1
Max tree depth                 : 2
Training time (sec)            : 0.015
Training accuracy              : 0.6135
Training log_loss              : 0.6588
Training auc                   : 0.6288



# Making predictions

Let's consider two positive and two negative examples **from the validation set** and see what the model predicts. We will do the following:
* Predict whether or not a loan is safe.
* Predict the probability that a loan is safe.

In [18]:
valid_safe_loans, valid_risky_loans = validation_data[validation_data[target] == 1], validation_data[validation_data[target] == -1]

sample_validation_data_risky, sample_validation_data_safe = valid_risky_loans[0:2], valid_safe_loans[0:2]
sample_validation_data = sample_validation_data_safe.append(sample_validation_data_risky)
len(sample_validation_data), sample_validation_data

(4,
 Columns:
 	grade	str
 	sub_grade	str
 	short_emp	int
 	emp_length_num	int
 	home_ownership	str
 	dti	float
 	purpose	str
 	term	str
 	last_delinq_none	int
 	last_major_derog_none	int
 	revol_util	float
 	total_rec_late_fee	float
 	safe_loans	int
 
 Rows: 4
 
 Data:
 +-------+-----------+-----------+----------------+----------------+-------+
 | grade | sub_grade | short_emp | emp_length_num | home_ownership |  dti  |
 +-------+-----------+-----------+----------------+----------------+-------+
 |   B   |     B3    |     0     |       11       |      OWN       | 11.18 |
 |   D   |     D1    |     0     |       10       |      RENT      | 16.85 |
 |   D   |     D2    |     0     |       3        |      RENT      | 13.97 |
 |   A   |     A5    |     0     |       11       |    MORTGAGE    | 16.33 |
 +-------+-----------+-----------+----------------+----------------+-------+
 +--------------------+------------+------------------+-----------------------+
 |      purpose       |    term  

## Explore label predictions

Now, we will use our model  to predict whether or not a loan is likely to default. For each row in the **sample_validation_data**, use the `dt_model` to predict whether or not the loan is classified as a **safe loan**. 

**Hint:** Be sure to use the `.predict()` method.

In [19]:
preds = dt_model.predict(sample_validation_data)
perc_correct_preds = (preds == sample_validation_data['safe_loans']).sum() / len(sample_validation_data)

preds, perc_correct_preds

(dtype: int
 Rows: 4
 [1, -1, -1, 1],
 0.5)

**Quiz Question:** What percentage of the predictions on `sample_validation_data` did `dt_model` get correct?
  - cf. above cell

## Explore probability predictions

For each row in the **sample_validation_data**, what is the probability (according **decision_tree_model**) of a loan being classified as **safe**? 


**Hint:** Set `output_type='probability'` to make **probability** predictions using **decision_tree_model** on `sample_validation_data`:

In [20]:
preds_proba = dt_model.predict(sample_validation_data, output_type='probability')
preds_proba

dtype: float
Rows: 4
[0.6532223224639893, 0.463798463344574, 0.356814444065094, 0.7621196508407593]

In [21]:
type(preds_proba)

turicreate.data_structures.sarray.SArray

In [22]:
import numpy as np
np.argmax(np.array(preds_proba))

3

**Quiz Question:** Which loan has the highest probability of being classified as a **safe loan**?
  - 4th one (at index 3 / start index is 0)

**Checkpoint:** Can you verify that for all the predictions with `probability >= 0.5`, the model predicted the label **+1**?

In [23]:
# yes as follows:
assert all(map(lambda t: t[0] == t[1], zip(preds_proba >= 0.5, preds == 1)))

In [24]:
preds_proba >= 0.5

dtype: int
Rows: 4
[1, 0, 0, 1]

In [25]:
# preds_proba.filter(lambda p: p >= 0.5)
turicreate.data_structures.sarray.SArray.where(preds_proba >= 0.5, preds_proba, 0)

dtype: float
Rows: 4
[0.6532223224639893, 0.0, 0.0, 0.7621196508407593]

### Tricky predictions!

Now, we will explore something pretty interesting. For each row in the **sample_validation_data**, what is the probability (according to `dt_small_model`) of a loan being classified as **safe**?

**Hint:** Set `output_type='probability'` to make **probability** predictions using `dt_small_model** on `sample_validation_data`:

In [26]:
small_preds = dt_small_model.predict(sample_validation_data)
small_preds

dtype: int
Rows: 4
[1, -1, -1, 1]

In [27]:
small_preds_proba = dt_small_model.predict(sample_validation_data, output_type='probability')
small_preds_proba

dtype: float
Rows: 4
[0.5803016424179077, 0.4085058867931366, 0.4085058867931366, 0.7454202175140381]

In [28]:
small_preds_proba[0], small_preds_proba[3] 

(0.5803016424179077, 0.7454202175140381)

**Quiz Question:** Notice that the probability preditions are the **exact same** for the 2nd and 3rd loans. Why would this happen?
  - same leaf

In [29]:
safe_loans_raw = loans[loans[target] == +1]
safe_loans_raw

grade,sub_grade,short_emp,emp_length_num,home_ownership,dti,purpose,term,last_delinq_none
B,B2,0,11,RENT,27.65,credit_card,36 months,1
C,C5,0,11,RENT,8.72,small_business,36 months,1
C,C1,0,11,RENT,20.0,other,36 months,0
A,A4,0,4,RENT,11.2,wedding,36 months,1
E,E1,0,10,RENT,5.35,car,36 months,1
C,C3,0,6,OWN,16.12,debt_consolidation,60 months,1
B,B5,0,11,OWN,10.78,debt_consolidation,36 months,1
B,B1,0,4,RENT,12.56,credit_card,36 months,1
D,D1,1,1,RENT,20.31,debt_consolidation,36 months,1
C,C4,0,5,RENT,18.6,home_improvement,36 months,0


# Evaluating accuracy of the decision tree model

Recall that the accuracy is defined as follows:
$$
\mbox{accuracy} = \frac{\mbox{# correctly classified examples}}{\mbox{# total examples}}
$$

Let us start by evaluating the accuracy of the `dt_small_model` and `dt_model` on the training data

In [30]:
_dt_small_acc = dt_small_model.evaluate(train_data)['accuracy']
_dt_acc = dt_model.evaluate(train_data)['accuracy']

assert _dt_small_acc < _dt_acc
_dt_small_acc, _dt_acc

(0.6135020416935311, 0.6405813453685794)

**Checkpoint:** You should see that the `dt_small_model` performs worse than the `dt_model` on the training data.

Now, let us evaluate the accuracy of the `dt_small_model` and `dt_model` on the entire **validation_data**, not just the subsample considered above.

In [31]:
dt_small_acc = dt_small_model.evaluate(validation_data)['accuracy']
dt_acc = dt_model.evaluate(validation_data)['accuracy']
dt_small_acc, dt_acc

(0.6193451098664369, 0.6367944851357173)

**Quiz Question:** What is the accuracy of `dt_model` on the validation set, rounded to the nearest .01?

In [32]:
round(dt_acc, 2)

0.64

## Evaluating accuracy of a complex decision tree model

Here, we will train a large decision tree with `max_depth=10`. This will allow the learned tree to become very deep, and result in a very complex model. Recall that in lecture, we prefer simpler models with similar predictive power. This will be an example of a more complicated model which has similar predictive power, i.e. something we don't want.

In [33]:
dt_big_model = turicreate.decision_tree_classifier.create(train_data, 
                                                          validation_set=None,
                                                          target=target, 
                                                          features=features, 
                                                          max_depth=10,
                                                          verbose=False)

dt_big_model.summary()

Class                          : DecisionTreeClassifier

Schema
------
Number of examples             : 37224
Number of feature columns      : 12
Number of unpacked features    : 12
Number of classes              : 2

Settings
--------
Number of trees                : 1
Max tree depth                 : 10
Training time (sec)            : 0.0341
Training accuracy              : 0.6655
Training log_loss              : 0.6068
Training auc                   : 0.7314



Now, let us evaluate **big_model** on the training set and validation set.

In [34]:
big_acc_train = dt_big_model.evaluate(train_data)['accuracy']
big_acc_valid = dt_big_model.evaluate(validation_data)['accuracy']
big_acc_train, big_acc_valid

(0.665538362346873, 0.6274235243429557)

**Checkpoint:** We should see that `dt_big_model` has even better performance on the training set than `dt_model` did on the training set.

In [35]:
## Yes:
assert big_acc_train > dt_model.evaluate(train_data)['accuracy']

**Quiz Question:** How does the performance of **big_model** on the validation set compare to **decision_tree_model** on the validation set? Is this a sign of overfitting?
  - not great (cf. below) - yes it is a sign of overfitting to the training dataset.

In [36]:
assert big_acc_valid <  dt_acc

### Quantifying the cost of mistakes

Every mistake the model makes costs money. In this section, we will try and quantify the cost of each mistake made by the model.

Assume the following:

* **False negatives**: Loans that were actually safe but were predicted to be risky. This results in an oppurtunity cost of losing a loan that would have otherwise been accepted. 
* **False positives**: Loans that were actually risky but were predicted to be safe. These are much more expensive because it results in a risky loan being given. 
* **Correct predictions**: All correct predictions don't typically incur any cost.


Let's write code that can compute the cost of mistakes made by the model. Complete the following 4 steps:
1. First, let us compute the predictions made by the model.
1. Second, compute the number of false positives.
2. Third, compute the number of false negatives.
3. Finally, compute the cost of mistakes made by the model by adding up the costs of true positives and false positives.

First, let us make predictions on `validation_data` using the `dt_model`:

In [37]:
preds = dt_model.predict(validation_data)

**False positives** are predictions where the model predicts +1 but the true label is -1. Complete the following code block for the number of false positives:

In [38]:
fp = len(list
    (filter(lambda t: t[0] == t[1] == 1, zip(preds == 1, validation_data['safe_loans'] == -1)))
)
fp

1656

**False negatives** are predictions where the model predicts -1 but the true label is +1. Complete the following code block for the number of false negatives:

In [39]:
fn = len(list
    (filter(lambda t: t[0] == t[1] == 1, zip(preds == -1, validation_data['safe_loans'] == 1)))
)
fn

1716

**Quiz Question:** Let us assume that each mistake costs money:
* Assume a cost of \$10,000 per false negative.
* Assume a cost of \$20,000 per false positive.

What is the total cost of mistakes made by `decision_tree_model` on `validation_data`?

In [40]:
fp * 20000 + fn * 10000
# 50,280,000

50280000